Here we'll first be loading the FC matrices and explore their structure

In [1]:
#using read_dataset from Datasets/FC/create_dataset.py to read the dataset
from Datasets.FC.create_dataset import read_dataset

In [2]:
dataset = read_dataset()

In [3]:
#ok, let's explore the data a bit more
#dataset is a list object of torch_geometric.data objects

#let's see the first element
print(dataset[0])

#it's a dictionary object, let's see the keys
print(dataset[0].keys())

#ok, let's see the values
print(dataset[0].values())

#it has 4 keys, 'x', 'edge_index', 'edge_attr' and 'y' where y=0 menas the patient is healthy and y=1 means the patient has Autism Spectrum Disorder (ASD)
"""graph = Data(x=ROI.reshape(-1,116).float(),
                     edge_index=G.indices().reshape(2,-1).long(),
                     edge_attr=G.values().reshape(-1,1).float(),
                     y=y.long())"""

#how much data do we have?
len(dataset)


Data(x=[116, 116], edge_index=[2, 1016], edge_attr=[1016, 1], y=[1])
['x', 'edge_index', 'edge_attr', 'y']
ValuesView({'x': tensor([[ 0.0000,  0.4543,  0.2477,  ...,  0.1753,  0.2247, -0.1751],
        [ 0.4543,  0.0000, -0.2204,  ..., -0.1947, -0.2258, -0.1434],
        [ 0.2477, -0.2204,  0.0000,  ..., -0.0521, -0.0804, -0.2025],
        ...,
        [ 0.1753, -0.1947, -0.0521,  ...,  0.0000,  0.6875, -0.1364],
        [ 0.2247, -0.2258, -0.0804,  ...,  0.6875,  0.0000,  0.0929],
        [-0.1751, -0.1434, -0.2025,  ..., -0.1364,  0.0929,  0.0000]]), 'edge_index': tensor([[  0,   0,   0,  ..., 114, 115, 115],
        [  1,  10,  12,  ..., 113,  94, 109]]), 'edge_attr': tensor([[0.4543],
        [0.5913],
        [0.4224],
        ...,
        [0.6875],
        [0.4846],
        [0.5437]]), 'y': tensor([0])})


1099

In [4]:
#set the seed
import torch
torch.manual_seed(0)

MODEL = "GAT"
DATASET = "FC"

from models.models_FC import GAT_framework as framework # import the model

gnn = framework(dataset)

print(gnn.model)
print(gnn.train_idx)

Net(
  (conv_layers): ModuleList(
    (0): GATConv(116, 128, heads=4)
    (1-4): 4 x GATConv(512, 128, heads=4)
  )
  (batch_norms): ModuleList(
    (0-4): 5 x BatchNorm(512)
  )
  (lin1): Linear(512, 128, bias=True)
  (lin2): Linear(128, 2, bias=True)
  (bn1): BatchNorm(128)
  (bn2): BatchNorm(2)
)
tensor([217, 137, 426,  ..., 643, 205, 434])


c:\Users\tomdu\miniconda3\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
# gnn.iterate()

/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch: 005, Loss: 0.255, Test Loss: 0.701, Train Acc: 0.619 Test Acc: 0.455
Epoch: 010, Loss: 0.169, Test Loss: 0.688, Train Acc: 0.959 Test Acc: 0.582
Epoch: 015, Loss: 0.142, Test Loss: 0.711, Train Acc: 0.999 Test Acc: 0.582
Epoch: 020, Loss: 0.128, Test Loss: 0.700, Train Acc: 0.998 Test Acc: 0.618
Epoch: 025, Loss: 0.123, Test Loss: 0.747, Train Acc: 1.000 Test Acc: 0.564
Epoch: 030, Loss: 0.119, Test Loss: 0.736, Train Acc: 0.999 Test Acc: 0.545
Epoch: 035, Loss: 0.103, Test Loss: 0.713, Train Acc: 1.000 Test Acc: 0.545
Epoch: 040, Loss: 0.097, Test Loss: 0.713, Train Acc: 1.000 Test Acc: 0.527
Epoch: 045, Loss: 0.092, Test Loss: 0.702, Train Acc: 1.000 Test Acc: 0.564
Epoch: 050, Loss: 0.089, Test Loss: 0.712, Train Acc: 1.000 Test Acc: 0.564
Epoch: 055, Loss: 0.087, Test Loss: 0.742, Train Acc: 1.000 Test Acc: 0.582
Epoch: 060, Loss: 0.088, Test Loss: 0.721, Train Acc: 0.999 Test Acc: 0.600
Epoch: 065, Loss: 0.080, Test Loss: 0.686, Train Acc: 1.000 Test Acc: 0.600
Epoch: 070, 

In [5]:
#gnn.train()

In [6]:
#save the model 
# gnn.save_model(path="models/"+DATASET+"_"+MODEL+"server2.pt")

Model saved in: models/FC_GATserver2.pt


In [6]:
#load the model
gnn.load_model(path="models/"+DATASET+"_"+MODEL+"server2.pt", map_location=torch.device('cpu'))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\tomdu\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tomdu\AppData\Local\Temp\ipykernel_18344\3648997669.py", line 2, in <module>
    gnn.load_model(path="models/"+DATASET+"_"+MODEL+"server2.pt", map_location=torch.device('cpu'))
  File "c:\Users\tomdu\OneDrive\Documents\ENSC_VU\4A-Vu\Thesis\Probing-GNN-representations\models\models_FC.py", line 1505, in load_model
    self.model.load_state_dict(torch.load(path, map_location=map_location))
  File "c:\Users\tomdu\miniconda3\lib\site-packages\torch\nn\modules\module.py", line 2041, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "conv_layers.0.lin_src.weight", "conv_layers.0.lin_dst.weight", "conv_layers.1.lin_src.weight", "conv_layers.1.lin_dst.

In [7]:
gnn.evaluate()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\tomdu\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tomdu\AppData\Local\Temp\ipykernel_20468\972427186.py", line 1, in <module>
    gnn.evaluate()
  File "c:\Users\tomdu\OneDrive\Documents\ENSC_VU\4A-Vu\Thesis\Probing-GNN-representations\models\models_FC.py", line 1509, in evaluate
    train_acc, train_loss = self.test(self.train_loader)
  File "c:\Users\tomdu\miniconda3\lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\tomdu\OneDrive\Documents\ENSC_VU\4A-Vu\Thesis\Probing-GNN-representations\models\models_FC.py", line 1485, in test
    out = self.model(data.x, data.edge_index, data.batch)
  File "c:\Users\tomdu\miniconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "c:\User

In [9]:
train_features, test_features = gnn.evaluate_with_features2()

In [10]:
print(len(train_features[0]))
len(train_features), len(test_features)

8


(1044, 55)

In [11]:
import networkx as nx
def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Alternative metrics for disconnected graphs
        # Option 1: Use the average path length of the largest connected component
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        density = nx.density(G)
        avg_path_len = calculate_avg_path_length(G)
        num_cliques = len(list(nx.find_cliques(G)))
        num_triangles = sum(nx.triangles(G).values()) / 3
        num_squares = sum(nx.square_clustering(G).values()) / 4
        number_of_node_in_the_largest_fully_connected_component = len(max(nx.connected_components(G), key=len))
        #small_world = nx.algorithms.smallworld.sigma(G)

        properties.append((num_nodes, num_edges, density, avg_path_len, num_cliques, num_triangles, num_squares, number_of_node_in_the_largest_fully_connected_component)) #, small_world))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties = compute_graph_properties(selected_dataset)

In [ ]:
print(len(train_properties))
train_properties[0:5]



In [13]:
import pickle as pkl
# save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "wb") as f:
    pkl.dump(train_properties, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "wb") as f:
    pkl.dump(test_properties, f)

#load the properties
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "rb") as f:
    train_properties = pkl.load(f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "rb") as f:
    test_properties = pkl.load(f)

In [14]:
"""
The embeddings of GCN are like this:
return F.log_softmax(x7, dim=-1), (x1, x2, x3, x4, x5, x_global, x6, x7)
"""


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
output_size = 1  # Predicting one property at a time
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Assume we have already evaluated to get features
# train_features, test_features = gnn.evaluate_with_features2()

# Extract x embeddings
train_x = np.array([feat[0] for feat in train_features])
test_x = np.array([feat[0] for feat in test_features])

# Extract 2, 3, 4, global, 5, 6, 7 embeddings
train_x2 = np.array([feat[1] for feat in train_features])
test_x2 = np.array([feat[1] for feat in test_features])

train_x3 = np.array([feat[2] for feat in train_features])
test_x3 = np.array([feat[2] for feat in test_features])

train_x4 = np.array([feat[3] for feat in train_features])
test_x4 = np.array([feat[3] for feat in test_features])

train_x5 = np.array([feat[4] for feat in train_features])
test_x5 = np.array([feat[4] for feat in test_features])

train_x_global = np.array([feat[5] for feat in train_features])
test_x_global = np.array([feat[5] for feat in test_features])

train_x6 = np.array([feat[6] for feat in train_features])
test_x6 = np.array([feat[6] for feat in test_features])

train_x7 = np.array([feat[7] for feat in train_features])
test_x7 = np.array([feat[7] for feat in test_features])

# Compute graph properties
# train_properties = compute_graph_properties(gnn.dataset[gnn.train_idx])
# test_properties = compute_graph_properties(gnn.dataset[gnn.test_idx])

# Convert to PyTorch tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
train_x2 = torch.tensor(train_x2, dtype=torch.float32)
train_x3 = torch.tensor(train_x3, dtype=torch.float32)
train_x4 = torch.tensor(train_x4, dtype=torch.float32)
train_x5 = torch.tensor(train_x5, dtype=torch.float32)
train_x_global = torch.tensor(train_x_global, dtype=torch.float32)
train_x6 = torch.tensor(train_x6, dtype=torch.float32)
train_x7 = torch.tensor(train_x7, dtype=torch.float32)

test_x = torch.tensor(test_x, dtype=torch.float32)
test_x2 = torch.tensor(test_x2, dtype=torch.float32)
test_x3 = torch.tensor(test_x3, dtype=torch.float32)
test_x4 = torch.tensor(test_x4, dtype=torch.float32)
test_x5 = torch.tensor(test_x5, dtype=torch.float32)
test_x_global = torch.tensor(test_x_global, dtype=torch.float32)
test_x6 = torch.tensor(test_x6, dtype=torch.float32)
test_x7 = torch.tensor(test_x7, dtype=torch.float32)

train_y = torch.tensor(train_properties, dtype=torch.float32)
test_y = torch.tensor(test_properties, dtype=torch.float32)

# Train and evaluate a model for each graph property and each embedding
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings = [(train_x, test_x), (train_x2, test_x2), (train_x3, test_x3), (train_x4, test_x4), (train_x5, test_x5), (train_x_global, test_x_global), (train_x6, test_x6), (train_x7, test_x7)]
embeddings_names = ['x', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "wb") as f:
    pkl.dump(results, f)

Epoch [1000/800000], Property: num_nodes, Loss: 6844.0854
Epoch [2000/800000], Property: num_nodes, Loss: 3919.4241
Epoch [3000/800000], Property: num_nodes, Loss: 2296.2935
Epoch [4000/800000], Property: num_nodes, Loss: 1381.7167
Epoch [5000/800000], Property: num_nodes, Loss: 870.4459
Epoch [6000/800000], Property: num_nodes, Loss: 585.7657
Epoch [7000/800000], Property: num_nodes, Loss: 421.5753
Epoch [8000/800000], Property: num_nodes, Loss: 318.8747
Epoch [9000/800000], Property: num_nodes, Loss: 249.0334
Epoch [10000/800000], Property: num_nodes, Loss: 199.0672
Epoch [11000/800000], Property: num_nodes, Loss: 162.2563
Epoch [12000/800000], Property: num_nodes, Loss: 134.5717
Epoch [13000/800000], Property: num_nodes, Loss: 113.4247
Epoch [14000/800000], Property: num_nodes, Loss: 97.1254
Epoch [15000/800000], Property: num_nodes, Loss: 84.5126
Epoch [16000/800000], Property: num_nodes, Loss: 74.7106
Epoch [17000/800000], Property: num_nodes, Loss: 67.0311
Epoch [18000/800000], P

Epoch [6000/800000], Property: num_edges, Loss: 131049.6250
Epoch [7000/800000], Property: num_edges, Loss: 116192.9219
Epoch [8000/800000], Property: num_edges, Loss: 103082.3125
Epoch [9000/800000], Property: num_edges, Loss: 91511.0469
Epoch [10000/800000], Property: num_edges, Loss: 81323.0703
Epoch [11000/800000], Property: num_edges, Loss: 72395.4844
Epoch [12000/800000], Property: num_edges, Loss: 64618.8516
Epoch [13000/800000], Property: num_edges, Loss: 57875.7031
Epoch [14000/800000], Property: num_edges, Loss: 52052.1445
Epoch [15000/800000], Property: num_edges, Loss: 47045.3867
Epoch [16000/800000], Property: num_edges, Loss: 42761.5156
Epoch [17000/800000], Property: num_edges, Loss: 39105.0195
Epoch [18000/800000], Property: num_edges, Loss: 35992.4453
Epoch [19000/800000], Property: num_edges, Loss: 33359.2578
Epoch [20000/800000], Property: num_edges, Loss: 31141.1133
Epoch [21000/800000], Property: num_edges, Loss: 29271.9355
Epoch [22000/800000], Property: num_edges

Epoch [142000/800000], Property: num_edges, Loss: 11487.2109
Epoch [143000/800000], Property: num_edges, Loss: 11479.0781
Epoch [144000/800000], Property: num_edges, Loss: 11471.1396
Epoch [145000/800000], Property: num_edges, Loss: 11463.3975
Epoch [146000/800000], Property: num_edges, Loss: 11455.8418
Epoch [147000/800000], Property: num_edges, Loss: 11448.4697
Epoch [148000/800000], Property: num_edges, Loss: 11441.2783
Epoch [149000/800000], Property: num_edges, Loss: 11434.2666
Epoch [150000/800000], Property: num_edges, Loss: 11427.4268
Epoch [151000/800000], Property: num_edges, Loss: 11420.7578
Epoch [152000/800000], Property: num_edges, Loss: 11414.2568
Epoch [153000/800000], Property: num_edges, Loss: 11407.9229
Epoch [154000/800000], Property: num_edges, Loss: 11401.7549
Epoch [155000/800000], Property: num_edges, Loss: 11395.7480
Epoch [156000/800000], Property: num_edges, Loss: 11389.8955
Epoch [157000/800000], Property: num_edges, Loss: 11384.1992
Epoch [158000/800000], P

Epoch [277000/800000], Property: num_edges, Loss: 11078.9902
Epoch [278000/800000], Property: num_edges, Loss: 11077.5850
Epoch [279000/800000], Property: num_edges, Loss: 11076.1875
Epoch [280000/800000], Property: num_edges, Loss: 11074.7998
Epoch [281000/800000], Property: num_edges, Loss: 11073.4199
Epoch [282000/800000], Property: num_edges, Loss: 11072.0469
Epoch [283000/800000], Property: num_edges, Loss: 11070.6846
Epoch [284000/800000], Property: num_edges, Loss: 11069.3311
Epoch [285000/800000], Property: num_edges, Loss: 11067.9863
Epoch [286000/800000], Property: num_edges, Loss: 11066.6514
Epoch [287000/800000], Property: num_edges, Loss: 11065.3242
Epoch [288000/800000], Property: num_edges, Loss: 11064.0049
Epoch [289000/800000], Property: num_edges, Loss: 11062.6963
Epoch [290000/800000], Property: num_edges, Loss: 11061.3955
Epoch [291000/800000], Property: num_edges, Loss: 11060.1035
Epoch [292000/800000], Property: num_edges, Loss: 11058.8184
Epoch [293000/800000], P

Epoch [412000/800000], Property: num_edges, Loss: 10969.7822
Epoch [413000/800000], Property: num_edges, Loss: 10969.5527
Epoch [414000/800000], Property: num_edges, Loss: 10969.3291
Epoch [415000/800000], Property: num_edges, Loss: 10969.1191
Epoch [416000/800000], Property: num_edges, Loss: 10968.9189
Epoch [417000/800000], Property: num_edges, Loss: 10968.7266
Epoch [418000/800000], Property: num_edges, Loss: 10968.5420
Epoch [419000/800000], Property: num_edges, Loss: 10968.3672
Epoch [420000/800000], Property: num_edges, Loss: 10968.1992
Epoch [421000/800000], Property: num_edges, Loss: 10968.0391
Epoch [422000/800000], Property: num_edges, Loss: 10967.8848
Epoch [423000/800000], Property: num_edges, Loss: 10967.7412
Epoch [424000/800000], Property: num_edges, Loss: 10967.6045
Epoch [425000/800000], Property: num_edges, Loss: 10967.4736
Epoch [426000/800000], Property: num_edges, Loss: 10967.3545
Epoch [427000/800000], Property: num_edges, Loss: 10967.2412
Epoch [428000/800000], P

...

Epoch [552000/800000], Property: num_triangles, Loss: 810319.6875
Epoch [553000/800000], Property: num_triangles, Loss: 809351.4375
Epoch [554000/800000], Property: num_triangles, Loss: 808385.2500
Epoch [555000/800000], Property: num_triangles, Loss: 807421.1250
Epoch [556000/800000], Property: num_triangles, Loss: 806459.1250
Epoch [557000/800000], Property: num_triangles, Loss: 805499.1875
Epoch [558000/800000], Property: num_triangles, Loss: 804541.3125
Epoch [559000/800000], Property: num_triangles, Loss: 803585.5000
Epoch [560000/800000], Property: num_triangles, Loss: 802631.8750
Epoch [561000/800000], Property: num_triangles, Loss: 801680.1250
Epoch [562000/800000], Property: num_triangles, Loss: 800730.6250
Epoch [563000/800000], Property: num_triangles, Loss: 799783.1250
Epoch [564000/800000], Property: num_triangles, Loss: 798837.6875
Epoch [565000/800000], Property: num_triangles, Loss: 797894.4375
Epoch [566000/800000], Property: num_triangles, Loss: 796953.1250
Epoch [567

Epoch [677000/800000], Property: num_triangles, Loss: 705381.0625
Epoch [678000/800000], Property: num_triangles, Loss: 704672.3125
Epoch [679000/800000], Property: num_triangles, Loss: 703965.7500
Epoch [680000/800000], Property: num_triangles, Loss: 703261.2500
Epoch [681000/800000], Property: num_triangles, Loss: 702558.6875
Epoch [682000/800000], Property: num_triangles, Loss: 701858.3750
Epoch [683000/800000], Property: num_triangles, Loss: 701160.0625
Epoch [684000/800000], Property: num_triangles, Loss: 700463.8750
Epoch [685000/800000], Property: num_triangles, Loss: 699769.6250
Epoch [686000/800000], Property: num_triangles, Loss: 699077.5625
Epoch [687000/800000], Property: num_triangles, Loss: 698387.5000
Epoch [688000/800000], Property: num_triangles, Loss: 697699.5625
Epoch [689000/800000], Property: num_triangles, Loss: 697013.7500
Epoch [690000/800000], Property: num_triangles, Loss: 696329.9375
Epoch [691000/800000], Property: num_triangles, Loss: 695648.2500
Epoch [692

Epoch [1000/800000], Property: num_squares, Loss: 27.0482
Epoch [2000/800000], Property: num_squares, Loss: 18.7173
Epoch [3000/800000], Property: num_squares, Loss: 12.3918
Epoch [4000/800000], Property: num_squares, Loss: 7.8061
Epoch [5000/800000], Property: num_squares, Loss: 4.7330
Epoch [6000/800000], Property: num_squares, Loss: 2.9268
Epoch [7000/800000], Property: num_squares, Loss: 2.0770
Epoch [8000/800000], Property: num_squares, Loss: 1.8007
Epoch [9000/800000], Property: num_squares, Loss: 1.7431
Epoch [10000/800000], Property: num_squares, Loss: 1.7297
Epoch [11000/800000], Property: num_squares, Loss: 1.7255
Epoch [12000/800000], Property: num_squares, Loss: 1.7248
Epoch [13000/800000], Property: num_squares, Loss: 1.7247
Epoch [14000/800000], Property: num_squares, Loss: 1.7247
Epoch [15000/800000], Property: num_squares, Loss: 1.7247
Early stopping at epoch 15884
Embedding: torch.Size([1044, 2])
Property: num_squares
  Train MSE: 1.7247, Test MSE: 1.5197
  Train R²: 0

Epoch [69000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1987.8815
Epoch [70000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1901.0975
Epoch [71000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1816.3962
Epoch [72000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1733.7699
Epoch [73000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1653.2252
Epoch [74000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1574.7573
Epoch [75000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1498.3680
Epoch [76000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1424.0547
Epoch [77000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1352.0359
Epoch [78000/800000], Property: number_of_node

Epoch [147000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 14.2014
Epoch [148000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 13.6718
Epoch [149000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 13.1630
Epoch [150000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.6751
Epoch [151000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.2080
Epoch [152000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.7617
Epoch [153000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.3363
Epoch [154000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 10.9316
Epoch [155000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 10.5475
Epoch [156000/800000], Property: number_of_nodes_in_the

In [ ]:
#load results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "rb") as f:
    results = pkl.load(f)

In [16]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save
plt.savefig("results/"+DATASET+"_"+MODEL+"test_R2.png")


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+'_'+MODEL+'train_R2.png')


#### Test with more properties

In [ ]:
import networkx as nx
import numpy as np

def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Use the average path length of the largest connected component for disconnected graphs
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    
def betweenness_centralization(G):
    n = len(G)
    betweenness = nx.betweenness_centrality(G)
    max_betweenness = max(betweenness.values())
    centralization = sum(max_betweenness - bet for bet in betweenness.values())
    if n > 2:
        centralization /= (n - 1) * (n - 2) / 2
    return centralization

def pagerank_centralization(G, alpha=0.85):
    n = len(G)
    pagerank = nx.pagerank(G, alpha=alpha)
    max_pagerank = max(pagerank.values())
    centralization = sum(max_pagerank - pr for pr in pagerank.values())
    if n > 1:
        centralization /= (n - 1)
    return centralization

def clustering_properties(G):
    average_clustering = nx.average_clustering(G)
    transitivity = nx.transitivity(G)
    return average_clustering, transitivity

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        
        # Number of nodes
        num_nodes = G.number_of_nodes()
        
        # Number of edges
        num_edges = G.number_of_edges()
        
        # Density
        density = nx.density(G)
        
        # Average Path Length
        avg_path_len = calculate_avg_path_length(G)
        
        # Diameter
        if nx.is_connected(G):
            diameter = nx.diameter(G)
        else:
            # Use the diameter of the largest connected component for disconnected graphs
            components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
            largest_component = max(components, key=len)
            diameter = nx.diameter(largest_component)
        
        # Radius
        if nx.is_connected(G):
            radius = nx.radius(G)
        else:
            radius = nx.radius(largest_component)
        
        # Clustering Coefficient
        clustering_coeff = nx.average_clustering(G)
        
        # Transitivity
        transitivity = nx.transitivity(G)
        
        # Assortativity
        assortativity = nx.degree_assortativity_coefficient(G)
        
        # Number of Cliques
        num_cliques = len(list(nx.find_cliques(G)))
        
        # Number of Triangles
        num_triangles = sum(nx.triangles(G).values()) / 3
        
        # Number of Squares (4-cycles)
        num_squares = sum(nx.square_clustering(G).values()) / 4
        
        # Size of the Largest Connected Component
        largest_component_size = len(max(nx.connected_components(G), key=len))
        
        # Average Degree
        degrees = [d for n, d in G.degree()]
        avg_degree = np.mean(degrees)
        
        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        avg_betweenness_centrality = np.mean(list(betweenness_centrality.values()))
        
        # Eigenvalues of the Adjacency Matrix (for spectral properties)
        eigenvalues = np.linalg.eigvals(nx.adjacency_matrix(G).todense())
        spectral_radius = max(eigenvalues)
        algebraic_connectivity = sorted(eigenvalues)[1]  # second smallest eigenvalue
        
        # Graph Laplacian Eigenvalues
        laplacian_eigenvalues = np.linalg.eigvals(nx.laplacian_matrix(G).todense())
        graph_energy = sum(abs(laplacian_eigenvalues))
        
        # Small-World-ness
        # Compare clustering coefficient and average path length with those of a random graph
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)
        random_clustering_coeff = nx.average_clustering(random_graph)
        random_avg_path_len = calculate_avg_path_length(random_graph)
        small_world_coefficient = (clustering_coeff / random_clustering_coeff) / (avg_path_len / random_avg_path_len)

        # Calculate Betweenness Centralization
        betweenness_cent = betweenness_centralization(G)
        print(f"Betweenness Centralization: {betweenness_cent}")

        # Calculate PageRank Centralization
        pagerank_cent = pagerank_centralization(G)
        print(f"PageRank Centralization: {pagerank_cent}")

        # Calculate Clustering properties
        avg_clustering, transitivity = clustering_properties(G)
        print(f"Average Clustering Coefficient: {avg_clustering}")
        print(f"Transitivity: {transitivity}")
        
        properties.append((
            num_nodes,
            num_edges,
            density,
            avg_path_len,
            diameter,
            radius,
            clustering_coeff,
            transitivity,
            assortativity,
            num_cliques,
            num_triangles,
            num_squares,
            largest_component_size,
            avg_degree,
            avg_betweenness_centrality,
            spectral_radius,
            algebraic_connectivity,
            graph_energy,
            small_world_coefficient
        ))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties_long = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties_long = compute_graph_properties(selected_dataset)

#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "wb") as f:
    pkl.dump(train_properties_long, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "wb") as f:
    pkl.dump(test_properties_long, f)

#load the properties
# with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "rb") as f:
#     train_properties_long = pkl.load(f)

# with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "rb") as f:
#     test_properties_long = pkl.load(f)

In [ ]:
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']
train_y_long = torch.tensor(train_properties_long, dtype=torch.float32)
test_y_long = torch.tensor(test_properties_long, dtype=torch.float32)
#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names_long):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y_long[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y_long[:, i].cpu().numpy()
            test_target = test_y_long[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv_long.pkl", "wb") as f:
    pkl.dump(results, f)

In [ ]:

colors_long = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_names_long)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/"+DATASET+"_"+MODEL+"_test_R2_plot_long.png', dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_names_long)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)

    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/"+DATASET+"_"+MODEL+"_train_R2_plot_long.png', dpi=300, bbox_inches='tight')